In [1]:
import getpass
import os
import sys
import asyncio
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage
from typing import Literal
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from py2neo import Graph
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import PromptTemplate
from langgraph.checkpoint.memory import MemorySaver

from pathlib import Path
project_root = Path(r"F:\bigmodel\meet-Pok-mon\4.KGqa\Pokemon-KGQA").resolve()
sys.path.insert(0, str(project_root))
from NER.ner_model import *
from KGsql.KGsql import KGQueryAgent
from RAG.langchaingraph.query import GraphRAG
from websearch.websearcher import WebSearcher

f:\Anaconda\envs\pokemon\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# llm = ChatOllama(
#     base_url = "127.0.0.1:11434",  # 注意：这里需要替换成自己本地启动的endpoint
#     model="qwen2.5:7b",
# )
llm = ChatOpenAI(
    model="Doubao-pro-256k-1.5",  # 模型名称（需与后端匹配）
    base_url="http://139.224.116.116:3000/v1",  # 本地或远程 API 地址
    api_key="sk-36oMlDApF5Nlg0v23014A4B69e864000944151Cd75D82076"  # 如果无需鉴权，可留空
)

In [3]:
g = Graph("bolt://localhost:7687", auth=("neo4j", "woshishamo630"))

In [4]:
class AgentState(MessagesState):
    next: str

In [5]:
def chat(state: AgentState):
    messages = state["messages"]
    model_response = llm.invoke(messages)
    # final_response = [HumanMessage(content=model_response.content, name="chatbot")]
    return {"messages": model_response}

In [6]:
kgsql_agent = KGQueryAgent()

加载已有模型
模型初始化完成 ......


In [8]:
def print_stream(stream):
    for sub_stream in stream:
        print(sub_stream)  # 就是上面的示例中非流式直接调用的全部信息
input_message = {"messages": ["拥有皮卡丘的角色中，有哪些是赤红的伙伴？"]}
print_stream(kgsql_agent.agent.stream(input_message, stream_mode="updates"))

{'agent': {'messages': [AIMessage(content='首先调用 get_entity 函数，提取问题中的实体。接着，根据提取的实体皮卡丘，调用 get_pokemon_owners 函数，得到拥有皮卡丘的角色。然后，针对每个拥有皮卡丘的角色，调用 get_person_partners 函数，查看是否为赤红的伙伴。', additional_kwargs={'tool_calls': [{'id': 'call_rbmjabnii8nxi56h1qcatg9e', 'function': {'arguments': ' {\n        "question": "拥有皮卡丘的角色中，有哪些是赤红的伙伴？"\n    }\n', 'name': 'get_entity'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 1496, 'total_tokens': 1622, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'doubao-1-5-pro-256k-250115', 'system_fingerprint': None, 'id': '02174393040956927e9140493fc1cd076b4fe1b883ff101e91f06', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-20d1fa40-936b-42d9-be8e-a15bbd342d8e-0', tool_calls=[{'name': 'get_entity', 'args': 

In [7]:
def kgsql_node(state: AgentState):
    result = kgsql_agent.agent.invoke(state)
    return {
        "messages": [
            HumanMessage(content=result["messages"][-1].content, name="kg_sqler")
        ]
    }


In [8]:
# 配置参数
graph_rag_config = {
    "artifacts_path": "F:\\bigmodel\\meet-Pok-mon\\4.KGqa\\Pokemon-KGQA\\RAG\\artifacts",
    "llm_config": {
        "model": "deepseek-ai/DeepSeek-V3",
        "base_url": "http://139.224.116.116:3000/v1",
        "api_key": "sk-36oMlDApF5Nlg0v23014A4B69e864000944151Cd75D82076"
    },
    "community_level": 0
}

# 初始化GraphRAG系统
graph_rag = GraphRAG(
    artifacts_path=graph_rag_config["artifacts_path"],
    llm_config=graph_rag_config["llm_config"],
    community_level=graph_rag_config["community_level"]
)

In [9]:
def graph_rager(state: AgentState):
    messages = state["messages"]

    response = graph_rag.query(messages) 
    final_response = [HumanMessage(content=response.content, name="graph_rager")]
    return {"messages": final_response}

In [10]:
searcher = WebSearcher()

已覆盖现有集合: test
创建新集合: test
创建索引: {'index_type': 'IVF_FLAT', 'metric_type': 'COSINE', 'params': {'nlist': 16}}


In [11]:
async def web_searcher(state: AgentState):
    """完全异步版本的搜索函数"""
    messages = state["messages"]
   
    # 直接使用await替代asyncio.run
    response = await searcher.search_and_generate(messages[0].content)
    return {"messages": [HumanMessage(content=response, name="web_searcher")]}


In [12]:
#构建代理节点
members = ["chat","kg_sqler","graph_rager","web_searcher"]
options = members + ["FINISH"]

In [13]:
# class Router(TypedDict):
#     """Worker to route to next. If no workers needed, route to FINISH"""

#     next: Literal[*options] # type: ignore
    
# 创建提示模板
prompt = ChatPromptTemplate.from_template("""
请严格按以下JSON格式回复，只包含next字段，值必须是'chat'或'kg_sqler'或'graph_rager'或'web_searcher'或'FINISH'：

{{
    "next": "FINISH"
}}

输入：{input}
""")

In [14]:
def supervisor(state: AgentState):
    system_prompt = (
        "你被指定为对话监督员，负责协调以下工作模块的协作：{members}\n\n"
        "各模块职能划分：\n"
        "- chat：自然语言交互模块\n"
            "  • 直接处理用户输入的自然语言响应\n"
        "- kg_sqler：宝可梦知识图谱查询模块\n"
            "  • 属性数据（种族值/进化链/特性）\n"
            "  • 角色关系（训练师/劲敌/团队）\n"
            "  • 地域情报（地点/道馆/栖息地）\n"
        "- graph_rager：宝可梦相关知识库\n"
            "  • 人物介绍（如人物事迹等）\n"
            "  • 社群发现（如道馆派系识别）\n"
            "  • 路径分析（角色关联路径追踪）\n"
            "  • 时序关联（赛事参与时间轴分析）\n\n"
        "- web_searcher：实时联网搜索模块\n"
            "  • 当问题涉及最新资讯、新闻或时效性内容时使用\n"
            "  • 当其他知识库无法提供准确答案时使用\n"
            "  • 可获取官方公告、赛事结果等实时信息\n"
            "  • 能查询宝可梦相关社区讨论和玩家反馈\n"
            "  • 可验证其他模块提供信息的时效性和准确性\n\n"
        "模块调用原则：\n"
            "1. 优先使用本地知识库(kg_sqler/graph_rager)回答已知的宝可梦知识\n"
            "2. 当问题涉及实时信息或本地知识不足时，调用web_searcher\n"
            "3. 请根据用户请求指定下一个执行模块。"
            "4. 每个模块执行后将返回任务结果及状态。\n"
        "执行流程规范：\n"
        "1. chat模块最多能调用一次\n"
        "2. 可以链式调用多个模块（如先用kg_sqler查询，再用web_searcher验证）\n"
        "3. 你可以不断调用上述的模块，当某个模块的结果不足以回答用户的问题时（如未查询到相关结果），你可以继续调用其他模块，直到用户问题得到回答。"
        "4. 当你任务完成时，才能返回FINISH终止符"
)

    messages = [{"role": "system", "content": system_prompt},] + state["messages"]

    chain = prompt | llm | JsonOutputParser()
    response = chain.invoke({"input": messages})

    next_ = response["next"]
    
    if next_ == "FINISH":
        next_ = END
    
    return {"next": next_}

In [15]:
builder = StateGraph(AgentState)

builder.add_node("supervisor", supervisor)
builder.add_node("chat", chat)
builder.add_node("kg_sqler", kgsql_node)
builder.add_node("graph_rager", graph_rager)
builder.add_node(
    "web_searcher",
    RunnableLambda(web_searcher)  
)

In [16]:
for member in members:
    # 我们希望我们的工人在完成工作后总是向主管“汇报”
    builder.add_edge(member, "supervisor")

In [17]:
#添加supervisor的条件边
builder.add_conditional_edges("supervisor", lambda state: state["next"])
# 添加开始和节点
builder.add_edge(START, "supervisor")
#增加检查点
memory = MemorySaver()
# 编译图
graph = builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [27]:
config = {"configurable": {"thread_id": "9"}}
async def print_stream_async(stream):
    chunks = []
    async for chunk in stream:
        chunks.append(chunk["messages"][-1])
    print(chunks[-1].content)

# 使用 astream 替代 stream
input_message = {"messages": ["皮卡丘是什么属性？"]}
stream = graph.astream(input_message, config, stream_mode="values")
await print_stream_async(stream)

皮卡丘的属性是电属性。


In [ ]:
config = {"configurable": {"thread_id": "3"}}
def print_stream(stream):
    for sub_stream in stream:
        print(sub_stream)  # 就是上面的示例中非流式直接调用的全部信息
input_message = {"messages": ["恭平有哪些宝可梦？"]}
print_stream(graph.stream(input_message, config, stream_mode="updates"))

# input_message = {"messages": ["你好，请问我刚刚问你关于皮卡丘的什么了？"]}
# print_stream(graph.stream(input_message, config, stream_mode="updates"))

In [19]:
for chunk in graph.stream({"messages": "你好，你是谁？"}, config,stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

你好，你是谁？
================================ Human Message =================================

你好，你是谁？
================================== Ai Message ==================================

我是豆包呀，能陪你畅快聊天，解答各类问题，不管是生活知识、学习疑惑，还是娱乐八卦等，都能为你提供帮助。
================================== Ai Message ==================================

我是豆包呀，能陪你畅快聊天，解答各类问题，不管是生活知识、学习疑惑，还是娱乐八卦等，都能为你提供帮助。
================================== Ai Message ==================================
================================== Ai Message ==================================
================================== Ai Message ==================================

我是豆包，旨在随时和你交流探讨，帮你解决知识疑问、陪你畅谈各种话题，你有什么想聊的，都能告诉我。
================================== Ai Message ==================================

我是豆包，旨在随时和你交流探讨，帮你解决知识疑问、陪你畅谈各种话题，你有什么想聊的，都能告诉我。
================================== Ai Message ==================================
================================== Ai Message =